## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.datasets import cifar10
import numpy as np
from sklearn.preprocessing import OneHotEncoder

TensorFlow 2.x selected.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7) 
    return X_train, X_test, mean, std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier = Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu')) #32,3,3,input_shape=(32,32,3),activation='relu'
classifier.add(Convolution2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D((2, 2)))
classifier.add(BatchNormalization())


classifier.add(Convolution2D(64, (3, 3), padding='same', activation='relu'))
classifier.add(Convolution2D(128, (3, 3), activation='relu'))
classifier.add(MaxPooling2D((2, 2)))
classifier.add(BatchNormalization())

classifier.add(Convolution2D(128, (3, 3), padding='same', activation='relu'))
classifier.add(Convolution2D(256, (3, 3), activation='relu'))
classifier.add(MaxPooling2D((2, 2)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())
# classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(512, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(
    x_train, y_train, batch_size=100, epochs=100, 
    validation_data=(x_test, y_test), shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 9s 175us/sample - loss: 1.2960 - accuracy: 0.5487 - val_loss: 1.0173 - val_accuracy: 0.6472
Epoch 2/100
50000/50000 [==============================] - 6s 124us/sample - loss: 0.7717 - accuracy: 0.7288 - val_loss: 0.8939 - val_accuracy: 0.6984
Epoch 3/100
50000/50000 [==============================] - 6s 124us/sample - loss: 0.5786 - accuracy: 0.7962 - val_loss: 0.7359 - val_accuracy: 0.7526
Epoch 4/100
50000/50000 [==============================] - 6s 124us/sample - loss: 0.4507 - accuracy: 0.8423 - val_loss: 0.7589 - val_accuracy: 0.7481
Epoch 5/100
50000/50000 [==============================] - 6s 124us/sample - loss: 0.3488 - accuracy: 0.8779 - val_loss: 0.7335 - val_accuracy: 0.7635
Epoch 6/100
50000/50000 [==============================] - 6s 123us/sample - loss: 0.2625 - accuracy: 0.9073 - val_loss: 0.7580 - val_accuracy: 0.7788
Epoch 7/100
50000/50000 [===================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example = (np.zeros(shape=(1, 32, 32, 3)) - mean_train) / (std_train + 1e-7) 
classifier.predict(input_example)

array([[6.4729536e-03, 1.7891666e-06, 7.6482786e-07, 9.4037598e-01,
        9.3654040e-03, 3.5250155e-06, 1.3629631e-04, 1.1122281e-06,
        4.3639291e-02, 2.8089191e-06]], dtype=float32)

In [6]:
classifier.evaluate(x_test, y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.719766922879219, 0.8188]